# **Mounting and Uploading Required File**

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

file_path = '/content/drive/My Drive/modelpred.csv'
df = pd.read_csv(file_path)

<ipython-input-2-90d26aeff370>:4: DtypeWarning: Columns (51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


**Using Naive Average Predicting Expected Runs On A Particular Ball**

In [3]:
# Ensure necessary columns are in the correct datatype
df['inns_rr'] = pd.to_numeric(df['inns_rr'], errors='coerce')
df['inns_balls_rem'] = pd.to_numeric(df['inns_balls_rem'], errors='coerce')
df['inns_wkts'] = pd.to_numeric(df['inns_wkts'], errors='coerce')
df['batruns'] = pd.to_numeric(df['batruns'], errors='coerce')

# Group by the unique combinations of 'inns_rr', 'inns_balls_rem', and 'inns_wkts'
grouped = df.groupby(['inns_rr', 'inns_balls_rem', 'inns_wkts'])['batruns'].mean().reset_index()

# Rename the 'batruns' column to 'XRuns' in the grouped DataFrame
grouped.rename(columns={'batruns': 'XRuns'}, inplace=True)

# Merge the grouped DataFrame back into the original DataFrame
df = df.merge(grouped, on=['inns_rr', 'inns_balls_rem', 'inns_wkts'], how='left')

# Display the first few rows of the updated DataFrame
print(df.head())

# Save the updated DataFrame back to CSV
new_file_path = '/content/drive/My Drive/ball_by_ball_expectedruns.csv'
df.to_csv(new_file_path, index=False)

print("The DataFrame with the 'XRuns' column has been successfully saved to:", new_file_path)

   p_match  inns          bat  p_bat   team_bat            bowl  p_bowl  \
0  1001349     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
1  1001349     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
2  1001349     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
3  1001351     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
4  1001351     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   

   team_bowl  ball  ball_id  ... wagonX  wagonY  wagonZone              line  \
0  Sri Lanka     1     0.01  ...    208     178          3     ON_THE_STUMPS   
1  Sri Lanka     2     0.02  ...    243     175          3     ON_THE_STUMPS   
2  Sri Lanka     3     0.03  ...    189     242          4     ON_THE_STUMPS   
3  Sri Lanka     3     0.03  ...    166     194          5  OUTSIDE_OFFSTUMP   
4  Sri Lanka     4     0.04  ...     15     222          6  OUTSIDE_OFFSTUMP   

                   length      shot  control  predscore  wprob     X

In [4]:

# Add a new column 'RAAR' which is the difference between 'batruns' and 'XRuns'
df['RAAR'] = df['batruns'] - df['XRuns']

# Display the first few rows of the updated DataFrame
print(df.head())

# Save the updated DataFrame back to CSV
new_file_path = '/content/drive/My Drive/ball_by_ball_expectedruns_with_RAAR.csv'
df.to_csv(new_file_path, index=False)

print("The DataFrame with the 'RAAR' column has been successfully saved to:", new_file_path)

   p_match  inns          bat  p_bat   team_bat            bowl  p_bowl  \
0  1001349     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
1  1001349     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
2  1001349     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
3  1001351     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   
4  1001351     1  Aaron Finch   5334  Australia  Lasith Malinga   49758   

   team_bowl  ball  ball_id  ... wagonY  wagonZone              line  \
0  Sri Lanka     1     0.01  ...    178          3     ON_THE_STUMPS   
1  Sri Lanka     2     0.02  ...    175          3     ON_THE_STUMPS   
2  Sri Lanka     3     0.03  ...    242          4     ON_THE_STUMPS   
3  Sri Lanka     3     0.03  ...    194          5  OUTSIDE_OFFSTUMP   
4  Sri Lanka     4     0.04  ...    222          6  OUTSIDE_OFFSTUMP   

                   length      shot  control  predscore  wprob     XRuns  \
0  SHORT_OF_A_GOOD_LENGTH  DEFENDED     

**RAAR (Impact Innings Filtration)**

In [5]:
import pandas as pd

# Load the data
df = pd.read_csv('/content/drive/My Drive/ball_by_ball_expectedruns_with_RAAR.csv', low_memory=False)

# Ensure RAAR column exists
if 'RAAR' not in df.columns:
    df['RAAR'] = df['batruns'] - df['XRuns']

# Step 1: Filter rows where wide is 0 and competition is T20I
df_filtered = df[(df['wide'] == 0)]

# Step 2: Further filter for Babar Azam's entries
babar_df = df_filtered[df_filtered['bat'] == 'Babar Azam']

# Step 3: Calculate total batruns and RAAR for each p_match
babar_summary = babar_df.groupby('p_match').agg(
    batruns_sum=('batruns', 'sum'),
    RAAR_sum=('RAAR', 'sum'),
    match_date=('date', 'first'),
    team_bat=('team_bat', 'first'),
    team_bowl=('team_bowl', 'first'),
    four_count=('outcome', lambda x: (x == 'four').sum()),
    six_count=('outcome', lambda x: (x == 'six').sum()),
    total_occurrences=('bat', 'count')
).reset_index()

# Step 4: Filter for innings where batruns_sum >= 100
babar_impact_innings = babar_summary[babar_summary['batruns_sum'] >= 100]

# Display the results
pd.set_option('display.max_rows', None)


In [6]:
print(babar_impact_innings)

# Save the result to a CSV file
output_file_path = '/content/drive/My Drive/babar_azam_impact_innings.csv'
babar_impact_innings.to_csv(output_file_path, index=False)

print("The filtered summary for Babar Azam has been successfully saved to:", output_file_path)

     p_match  batruns_sum   RAAR_sum  match_date                   team_bat  \
80   1167193          102  24.187067  2019-08-09                   Somerset   
97   1169537          101   4.301562  2019-05-01                   Pakistan   
110  1207765          114  34.627263  2020-09-16                   Somerset   
151  1251577          122  12.136325  2021-04-14                   Pakistan   
165  1277318          105  15.231873  2021-09-30  Central Punjab (Pakistan)   
193  1327229          110 -11.063816  2022-09-22                   Pakistan   
206  1339615          101  17.485561  2023-04-15                   Pakistan   
216  1354948          115  -6.815850  2023-03-08             Peshawar Zalmi   
223  1382887          104   1.245982  2023-08-07           Colombo Strikers   
249  1416484          112   9.378025  2024-02-26             Peshawar Zalmi   

               team_bowl  four_count  six_count  total_occurrences  
80             Hampshire           7          6              

In [12]:
import plotly.graph_objects as go

# Create an interactive table using Plotly
fig_table = go.Figure(data=[go.Table(
    header=dict(values=list(babar_impact_innings),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[babar_impact_innings[col] for col in babar_impact_innings],
               fill_color='lavender',
               align='left'))
])

# Update layout for better readability
fig_table.update_layout(
    title="Babar Azam Centuries In Context Of The Game",
    height=600,  # You can adjust the height as needed
    width=1000,  # You can adjust the width as needed
)

# Show the table
fig_table.show()


In [13]:
import pandas as pd
import plotly.graph_objects as go

# Function to load the data
filepath = '/content/drive/My Drive/ball_by_ball_expectedruns_with_RAAR.csv'
def load_data(filepath):
    # Load the CSV file
    df = pd.read_csv(filepath, low_memory=False)

**Babar Azam in Powerplay Contextual Performance**

In [20]:
import pandas as pd

# Ensure RAAR column exists
if 'RAAR' not in df.columns:
    df['RAAR'] = df['batruns'] - df['XRuns']

# Filter rows for Babar Azam and relevant innings
babar_df = df[(df['bat'] == 'Babar Azam') & (df['inns_balls_rem'].between(84, 120))]

# List of years to process
years = [2020, 2021, 2022, 2023, 2024]
results = []

# Process each year separately
for year in years:
    # Filter data for the specific year
    year_df = babar_df[babar_df['year'] == year]

    # Aggregate data for the filtered year
    aggregated = year_df.groupby('year').agg(
        batruns_total=('batruns', 'sum'),
        RAAR_total=('RAAR', 'sum'),
        four_count=('outcome', lambda x: (x == 'four').sum()),
        six_count=('outcome', lambda x: (x == 'six').sum())
    ).reset_index()

    # Add aggregated results to the list
    results.append(aggregated)

# Concatenate results into a single DataFrame
results_df = pd.concat(results, ignore_index=True)

# Display the results
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
print(results_df)

import plotly.graph_objects as go

# Create an interactive table using Plotly
fig_table = go.Figure(data=[go.Table(
    header=dict(values=list(results_df),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[results_df[col] for col in results_df],
               fill_color='lavender',
               align='left'))
])

# Update layout for better readability
fig_table.update_layout(
    title="Babar Azam Contextual Performance in T20s in Powerplay Over The Years",
    height=600,  # You can adjust the height as needed
    width=1000,  # You can adjust the width as needed
)

# Show the table
fig_table.show()



   year  batruns_total  RAAR_total  four_count  six_count
0  2020            466  -18.957989          68          2
1  2021            512    8.314751          68          5
2  2022            514  -18.911343          75          2
3  2023            410  -53.495449          63          5
4  2024            643  -18.520476          92         10


**Babar Azam In Powerplay Overall T20s**

In [17]:
import pandas as pd
# Ensure RAAR column exists
if 'RAAR' not in df.columns:
    df['RAAR'] = df['batruns'] - df['XRuns']

# Filter rows for Babar Azam and relevant innings
babar_df = df[(df['bat'] == 'Babar Azam') & (df['inns_balls_rem'].between(84, 120))]

# Calculate the totals directly
batruns_total = babar_df['batruns'].sum()
RAAR_total = babar_df['RAAR'].sum()
four_count = babar_df['outcome'].apply(lambda x: x == 'four').sum()
six_count = babar_df['outcome'].apply(lambda x: x == 'six').sum()
total_occurrences = babar_df['bat'].count()

# Create a DataFrame for easy display and saving
aggregated_df = pd.DataFrame({
    'batruns_total': [batruns_total],
    'RAAR_total': [RAAR_total],
    'four_count': [four_count],
    'six_count': [six_count],
    'total_occurrences': [total_occurrences]
})

# Display the results
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
print(aggregated_df)
import plotly.graph_objects as go

# Create an interactive table using Plotly
fig_table = go.Figure(data=[go.Table(
    header=dict(values=list(aggregated_df),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[aggregated_df[col] for col in aggregated_df],
               fill_color='lavender',
               align='left'))
])

# Update layout for better readability
fig_table.update_layout(
    title="Babar Azam In Powerplay All T20s In Context Of The Game",
    height=600,  # You can adjust the height as needed
    width=1000,  # You can adjust the width as needed
)

# Show the table
fig_table.show()



   batruns_total  RAAR_total  four_count  six_count  total_occurrences
0           3491 -159.081367         495         32               2979


**Babar Azam in Powerplay in T20Is Contextual Performance**

In [19]:
import pandas as pd

# Ensure RAAR column exists
if 'RAAR' not in df.columns:
    df['RAAR'] = df['batruns'] - df['XRuns']

# Filter rows for Babar Azam, relevant innings, and T20I competition
babar_df = df[(df['bat'] == 'Babar Azam') & (df['inns_balls_rem'].between(84, 120)) & (df['competition'] == 'T20I')]

# List of years to process
years = [2020, 2021, 2022, 2023, 2024]
results = []

# Process each year separately
for year in years:
    # Filter data for the specific year
    year_df = babar_df[babar_df['year'] == year]

    # Aggregate data for the filtered year
    aggregated = year_df.groupby('year').agg(
        batruns_total=('batruns', 'sum'),
        RAAR_total=('RAAR', 'sum'),
        four_count=('outcome', lambda x: (x == 'four').sum()),
        six_count=('outcome', lambda x: (x == 'six').sum()),
        total_occurrences=('bat', 'count')
    ).reset_index()

    # Add aggregated results to the list
    results.append(aggregated)

# Concatenate results into a single DataFrame
results_df = pd.concat(results, ignore_index=True)

# Display the results
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
print(results_df)

import plotly.graph_objects as go

# Create an interactive table using Plotly
fig_table = go.Figure(data=[go.Table(
    header=dict(values=list(results_df),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[results_df[col] for col in results_df],
               fill_color='lavender',
               align='left'))
])

# Update layout for better readability
fig_table.update_layout(
    title="Babar Azam in Powerplay in T20Is Contextual Performance Over The Years",
    height=600,  # You can adjust the height as needed
    width=1000,  # You can adjust the width as needed
)

# Show the table
fig_table.show()


   year  batruns_total  RAAR_total  four_count  six_count  total_occurrences
0  2020            115   12.489366          17          0                 84
1  2021            264   -5.430979          37          1                248
2  2022            370  -15.611447          54          1                327
3  2023             55    0.199916           9          0                 48
4  2024            248   -6.104250          35          4                219


**Innings Filration Of Big T20 Players** _If it is Failure, Cameo, Impact Innings or Scoreboard Pressure Creating Innings_

In [23]:
import pandas as pd
import plotly.graph_objects as go

# Step 1: Add RAAR column if it doesn't already exist
if 'RAAR' not in df.columns:
    df['RAAR'] = df['batruns'] - df['XRuns']

# Step 2: Filter rows where wide is 0
df_filtered = df[df['wide'] == 0]

# Step 3: Filter only the relevant batsmen
batsmen = [
    "Babar Azam", "Suryakumar Yadav", "Virat Kohli", "Rohit Sharma",
    "Mohammad Rizwan", "Devon Conway", "Travis Head", "Phill Salt",
    "Nicholas Pooran", "Jos Buttler", "Andre Russell", "Fakhar Zaman"
]
df_filtered = df_filtered[df_filtered['bat'].isin(batsmen)]

# Step 4: Calculate total sum of batruns and RAAR for each bat in each p_match
summary = df_filtered.groupby(['p_match', 'bat']).agg({
    'batruns': 'sum',
    'RAAR': 'sum',
    'bat': 'count'
}).rename(columns={'bat': 'entries'}).reset_index()

# Step 5: Label the innings
def label_innings(row):
    if row['entries'] < 12:
        if row['RAAR'] < 0:
            return 'Failure'
        else:
            return 'Cameo'
    else:
        if row['RAAR'] > 0:
            return 'Impact innings'
        else:
            return 'Scoreboard pressure creating innings'

summary['innings_label'] = summary.apply(label_innings, axis=1)

# Step 6: Loop through each batsman and create separate Plotly tables
for batsman in batsmen:
    batsman_data = summary[summary['bat'] == batsman]

    # Create a Plotly table for each batsman
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(batsman_data.columns),
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[batsman_data[col] for col in batsman_data.columns],
                   fill_color='lavender',
                   align='left'))
    ])

    # Show the table for each batsman
    fig.update_layout(title=f"Innings Summary for {batsman}")
    fig.show()

**Filtered Percentage of Each Batter**

In [31]:
import plotly.graph_objects as go
import numpy as np

# Step 7: Calculate the percentage of each innings type for each batsman (same as before)
percentage_summary = summary.groupby(['bat', 'innings_label']).agg({
    'p_match': 'count'
}).rename(columns={'p_match': 'count'}).reset_index()

# Calculate total innings per batsman
total_innings = summary.groupby('bat').agg({
    'p_match': 'count'
}).rename(columns={'p_match': 'total_innings'}).reset_index()

# Merge total innings with percentage summary
percentage_summary = pd.merge(percentage_summary, total_innings, on='bat')

# Calculate percentage of each innings type
percentage_summary['percentage'] = (percentage_summary['count'] / percentage_summary['total_innings']) * 100

# Step 8: Pivot the data to create a table where rows are batsmen and columns are innings types
pivot_table = percentage_summary.pivot(index='bat', columns='innings_label', values='percentage').fillna(0)

# Step 9: Prepare to plot each column separately with a different color scale
batsmen = pivot_table.index
innings_types = pivot_table.columns

# Define different color scales for each innings type
color_scales = ['Blues', 'Greens', 'Reds', 'Oranges']

# Create subplots with shared Y-axis for batsmen
fig = go.Figure()

# Step 10: Loop through each innings type and create separate heatmaps
for i, innings_type in enumerate(innings_types):
    # Add heatmap trace for each innings type with its own color scale
    fig.add_trace(go.Heatmap(
        z=np.array(pivot_table[innings_type]).reshape(-1, 1),  # Data for this innings type
        x=[innings_type],  # Label the column
        y=batsmen,  # Batsmen as rows
        colorscale=color_scales[i % len(color_scales)],  # Use a different color scale for each innings type
        colorbar=dict(title=f"{innings_type} %", len=0.3, y=1 - (i * 0.2))  # Position colorbar for each column
    ))

# Update layout to make sure subplots align properly
fig.update_layout(
    title="Heatmap of Innings Type Percentages with Separate Color Theories for Each Column",
    xaxis_title="Innings Type",
    yaxis_title="Batsman",
    height=600,
    width=1000,
    showlegend=False
)

# Show the combined heatmap with distinct color scales
fig.show()


In [32]:
import plotly.express as px

# Step 7: Calculate the percentage of each innings type for each batsman (same as before)
percentage_summary = summary.groupby(['bat', 'innings_label']).agg({
    'p_match': 'count'
}).rename(columns={'p_match': 'count'}).reset_index()

# Calculate total innings per batsman
total_innings = summary.groupby('bat').agg({
    'p_match': 'count'
}).rename(columns={'p_match': 'total_innings'}).reset_index()

# Merge total innings with percentage summary
percentage_summary = pd.merge(percentage_summary, total_innings, on='bat')

# Calculate percentage of each innings type
percentage_summary['percentage'] = (percentage_summary['count'] / percentage_summary['total_innings']) * 100

# Step 8: Pivot the data to create a table where rows are batsmen and columns are innings types
pivot_table = percentage_summary.pivot(index='bat', columns='innings_label', values='percentage').fillna(0)

# Step 9: Create a heatmap
fig = px.imshow(
    pivot_table,
    labels=dict(x="Innings Type", y="Batsman", color="Percentage"),
    x=pivot_table.columns,
    y=pivot_table.index,
    text_auto=True,  # Show the actual values inside the heatmap cells
    aspect="auto"    # Adjust aspect ratio
)

# Update layout to improve readability
fig.update_layout(
    title="Heatmap of Innings Type Percentages for Each Batsman",
    xaxis_title="Innings Type",
    yaxis_title="Batsman",
    coloraxis_colorbar=dict(title="Percentage"),
)

# Show the heatmap
fig.show()
